Name: Kalaiarasi sargunaraj 
Student No: 17200399
Name:Parkavi Kolandavel Mani
Student No: 18200072



In [1]:
import urllib.request
import bs4
import pandas as pd
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re
from sklearn.model_selection import KFold
import string
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kalaisr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kalaisr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Creating empty dictionaries to save the reviews and respective class labels

In [2]:
baseURL = "http://mlg.ucd.ie/modules/yalp/"
reviews_automative={'class':[], "reviews":[]}
reviews_restaurants={'class':[], "reviews":[]}

method that accepts a link as param and returns the source of the link's web page

In [3]:
def getWebPage(relative_link):
    link = baseURL+relative_link
    response = urllib.request.urlopen(link)
    if response.code==200:
        return response.read().decode()
    else:
        print("Request failed with a code"+response.code)

Method that accepts a link as  param, scrapes and returns the reviews and respective ratings. It stores the scraped reviews in a dictionary. It also scrapes the ratings associated with the review and converts that to one of the labels - 'positive','negative' before returning them

In [4]:
def getReviews(relative_link):
    html = getWebPage(relative_link)
    parser = bs4.BeautifulSoup(html,"html.parser")
    reviews={'class':[], "reviews":[]}
    for business in parser.find_all("a", href=True):
     business_link = business['href']
     business_html = getWebPage(business_link)
     review_parser = bs4.BeautifulSoup(business_html,"html.parser")
     for review in review_parser.find_all(class_="review"):
        rating = review.find(class_="stars").find("img").get("alt")
        reviewText = review.find(class_="text").text
        reviews["class"].append('negative' if int(rating[0])<4 else 'positive')
        reviews["reviews"].append(reviewText.strip())
    return reviews

Storing the retrieved of both categories  in seperate data frame for the ease of manipulation.
Category A - Automotive
Category B - Restaurants

In [5]:
reviews_automative = getReviews("automotive_list.html")
reviews_restaurants = getReviews("restaurants_list.html")
columns =["class","reviews"]
reviews_automative_frame = pd.DataFrame.from_dict(reviews_automative)
reviews_restaurants_frame = pd.DataFrame.from_dict(reviews_restaurants)

Preprocessing the reviews before they are tokenized:
    1) removing all the numbers and punctuations as they are not of value on mining the documents in this context
    2)removing stop words- standard english stop words from the library along with some custom stop words
    3)removing words with length lesser than 2
    4) converting all terms to lower case

In [6]:
my_additional_stop_words = ["am","pm","and","the"]
stop_words = (text.ENGLISH_STOP_WORDS.union(my_additional_stop_words))
def review_preprocessor(tokens):
    r = re.sub('[\d]+', '', tokens.lower())
    r = "".join([ch for ch in r if ch not in string.punctuation]) 
    r = ' '.join([word for word in r.split() if word not in (stop_words)])
    r = ' '.join([word for word in r.split() if len(word)>2])
    return r



Definig a Lemmentizing Tokenizer

In [7]:
class LemmaTokenizer(object):
    def __init__(self):
         self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        a = [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
        return a

A method to vectorize the given corpus. It is built using a preprocessor that removes numbers, punctuations, terms with length less than 2 and stop words. Lemmentization has been done post tokenization. PArams has been tuned for specified the minimum and maximum term frequencies. ngram_range has been specied in range 1 to 2 to and is found to give good performance. 


In [8]:
def vectorize(reviews):
    vectorizer = CountVectorizer(preprocessor=review_preprocessor, tokenizer=LemmaTokenizer(), min_df=0.04, max_df=0.8, ngram_range=(1, 2), lowercase=False)
    return vectorizer.fit_transform(reviews)




Concatenating the reviews from both category data frames and vectorizing them togeather. They are seperated into two different matrices post vectorization. 

In [9]:
cat_A_len = len(reviews_automative_frame['reviews'])
cat_B_len  = len(reviews_restaurants_frame['reviews'])
new_list = reviews_automative_frame['reviews'].append(reviews_restaurants_frame['reviews'])
matrix = vectorize(new_list)
reviews_automative_matrix = matrix[:cat_A_len]
reviews_restaurants_matrix = matrix[cat_A_len:cat_A_len+cat_B_len]

Experiment 1: Support Vector Classification trained and tested on Category A - Automotive data
The accuracy was the highest for this category and classifier combination. This was expected as more data is available for category A when compared to category B. The Accuracy and Confusion martices have been printed.

In [10]:
data_train, data_test, target_train, target_test = train_test_split(reviews_automative_matrix, reviews_automative_frame["class"], test_size=0.3)
svc_clf = svm.SVC(kernel='linear', C=1)
svc_model = svm.SVC(kernel='linear', C=1)
svc_model.fit(data_train, target_train)
predicted = svc_model.predict(data_test)
print(accuracy_score(target_test, predicted))
cm = confusion_matrix(target_test, predicted, labels=['positive','negative'])
print(cm)

0.8604118993135011
[[267  26]
 [ 35 109]]


Experiment 1 evaluated with cross validation 

In [11]:
svc_clf = svm.SVC(kernel='linear', C=1)
acc_scores = cross_validate (svc_clf, reviews_automative_matrix, reviews_automative_frame["class"], cv=5,scoring='accuracy')
print(acc_scores['test_score'].mean())

0.8439989968297873


Experiment 2: Multi-layer Perceptron classifier trained and tested on Category B - Restaurants data. The accuracy was the slightly lower than the accuracy of SVC trained and tested with Category A data. The Accuracy and Confusion martices have been printed.

In [12]:
#data_train, data_test, target_train, target_test = train_test_split(reviews_automative_matrix, reviews_automative_frame["class"], test_size=0.4)
mlp_clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1)
scores = cross_val_score(mlp_clf, reviews_restaurants_matrix, reviews_restaurants_frame["class"], cv=5,
scoring="accuracy")
print(scores.mean())

0.782578991663619


Experiment 3: Support Vector Classification trained with Category A(Automotive data) and tested with Category B data(Restaurants). This surprisingly had a good accuracy which indicates that the trained model was able to generalise and make good predictions on other category data.

In [13]:
data_train, data_test, target_train, target_test = train_test_split(reviews_automative_matrix, reviews_automative_frame["class"], test_size=0.3)
r_data_train, r_data_test, r_target_train, r_target_test = train_test_split(reviews_restaurants_matrix, reviews_restaurants_frame["class"], test_size=0.4)
svc_model = svm.SVC(kernel='linear', C=1)
svc_model.fit(data_train, target_train)
predicted = svc_model.predict(r_data_test)
print(accuracy_score(r_target_test, predicted))
cm = confusion_matrix(r_target_test, predicted, labels=['positive','negative'])
print(cm)

0.7309027777777778
[[283  88]
 [ 67 138]]


Experiment 4: Support Vector Classification trained with Category B data(Restaurants) and tested with Category A(Automotive data). This also had a good accuracy, slightly lesser than the model trained with Category A data in Experiment 3 as expected.

In [14]:
data_train, data_test, target_train, target_test = train_test_split(reviews_automative_matrix, reviews_automative_frame["class"], test_size=0.3)
r_data_train, r_data_test, r_target_train, r_target_test = train_test_split(reviews_restaurants_matrix, reviews_restaurants_frame["class"], test_size=0.4)
svc_model = svm.SVC(kernel='linear', C=1)
svc_model.fit(r_data_train, r_target_train)
predicted = svc_model.predict(data_test)
print(accuracy_score(target_test, predicted))
cm = confusion_matrix(target_test, predicted, labels=['positive','negative'])
print(cm)

0.7208237986270023
[[226  63]
 [ 59  89]]


Experiment 5: Multi-layer Perceptron classifier trained with Category A(Automotive data) and tested with Category B data(Restaurants). This model also  gave a surprisingly good accuracy which indicates that the trained model was able to generalise and make good predictions on other category data.

In [15]:
data_train, data_test, target_train, target_test = train_test_split(reviews_automative_matrix, reviews_automative_frame["class"], test_size=0.3)
r_data_train, r_data_test, r_target_train, r_target_test = train_test_split(reviews_restaurants_matrix, reviews_restaurants_frame["class"], test_size=0.4)
mlp_clf_cross = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1)
mlp_clf_cross.fit(data_train, target_train)
predicted = mlp_clf_cross.predict(r_data_test)
print(accuracy_score(r_target_test, predicted))
cm = confusion_matrix(r_target_test, predicted, labels=['positive','negative'])
print(cm)

0.7482638888888888
[[300  72]
 [ 73 131]]


Experiment 6: Multi-layer Perceptron classifier trained with Category B data(Restaurants)  and tested with Category A(Automotive data)

In [16]:
data_train, data_test, target_train, target_test = train_test_split(reviews_automative_matrix, reviews_automative_frame["class"], test_size=0.3)
r_data_train, r_data_test, r_target_train, r_target_test = train_test_split(reviews_restaurants_matrix, reviews_restaurants_frame["class"], test_size=0.4)
mlp_clf_cross = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1)
mlp_clf_cross.fit(r_data_train, r_target_train)
predicted = mlp_clf_cross.predict(data_test)
print(accuracy_score(target_test, predicted))
cm = confusion_matrix(target_test, predicted, labels=['positive','negative'])
print(cm)

0.7780320366132724
[[225  70]
 [ 27 115]]


Analysis of Results:

It was Observed that Pre-Processing and vectorization of the reviews has a huge impact on the classification performance. Producing a Quality set of dimentions(terms) was vital in producing quality results. 
The classification of Category A - Automotive was exoected to have better accuracy than that of Category B - Restaurants  due to availability of more data. Hence results of Experiment 2 and 3 were as expected. 
Support Vector Classification performed better on both Categories when compared to Multi-layer Perceptron classifier .
When training the data on one category and testing it with the other category, it is expected to get a lesser accuracy and this was the case. Although,  experiments 3, 4, 5 and 6 showed that both the classifiers generalised well and gave a good accuracy only slighlt less than the accuracies that resulted from Experiment 2 and 3